In [1]:
import numpy as np
import torch
import torch.nn.functional as F
from torch import nn

In [2]:
# open text file and read in data as `text`
with open("Юдковски Элиезер. Гарри Поттер и методы рационального мышления - royallib.com.txt", "r") as f:
    text = f.read()

In [3]:
text[:100]

'Гарри Поттер и методы рационального мышления\n\n\n\n\nМы рады представить вам электронную версию этой зам'

In [4]:
# we create two dictionaries:
# 1. int2char, which maps integers to characters
# 2. char2int, which maps characters to unique integers
chars = tuple(set(text))

int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

# encode the text
# int2char = dict(enumerate(chars))
# char2int = {ch: ii for ii, ch in int2char.items()}

# encode the text
encoded = np.array([char2int[ch] for ch in text])

In [5]:
with open("anna.txt", "r") as f:
    text2 = f.read()
chars2 = tuple(set(text2))

int2char2 = dict(enumerate(chars2))
char2int2 = {ch: ii for ii, ch in int2char2.items()}

# encode the text
# int2char = dict(enumerate(chars))
# char2int = {ch: ii for ii, ch in int2char.items()}

# encode the text
encoded2 = np.array([char2int2[ch] for ch in text2])

In [6]:
def one_hot_encode(arr, n_labels):

    # Initialize the the encoded array
    one_hot = np.zeros((arr.size, n_labels), dtype=np.float32)

    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.0

    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))

    return one_hot

In [7]:
# check that the function works as expected
test_seq = np.array([[3, 5, 1]])
one_hot = one_hot_encode(test_seq, 8)

print(one_hot)

[[[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]]]


In [8]:
def get_batches(int_words, batch_size, seq_length):
    """
    Generates batches from encoded sequence.
    :param int_words: tensor of ints, which represents the text; shape: [batch_size, -1]
    :param batch_size: number of sequences per batch
    :param seq_length: number of encoded chars in a sequence
    :return: generator of pairs (x, y); x_shape, y_shape: [batch_size, seq_length - 1]
    """
    # 1. Truncate text, so there are only full batches
    window_size = seq_length + 1
    batch_size_total = batch_size * window_size
    n_batches = len(int_words) // batch_size_total
    int_words = int_words[: n_batches * batch_size_total]

    # 2. Reshape into batch_size rows
    int_words = int_words.reshape((batch_size, -1))

    # 3. Iterate through the text matrix
    for position in range(0, int_words.shape[1], window_size):
        x = int_words[:, position : position + window_size - 1]
        y = int_words[:, position + 1 : position + window_size]
        yield x, y

In [9]:
batches = get_batches(encoded, 8, 50)
# batches = get_batches(encoded2, 8, 50)
x, y = next(batches)
# print(x)
# print(y)

In [10]:
class CharRNN(nn.Module):
    def __init__(self, tokens, n_hidden=256, n_layers=2, drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr

        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}

        ## TODO: define the LSTM
        self.lstm = nn.LSTM(
            len(self.chars), n_hidden, n_layers, dropout=drop_prob, batch_first=True
        )

        ## TODO: define a dropout layer
        self.dropout = nn.Dropout(drop_prob)

        ## TODO: define the final, fully-connected output layer
        self.fc = nn.Linear(n_hidden, len(self.chars))

    def forward(self, x, hidden):
        """Forward pass through the network.
        These inputs are x, and the hidden/cell state `hidden`."""

        ## TODO: Get the outputs and the new hidden state from the lstm
        r_output, hidden = self.lstm(x, hidden)

        ## TODO: pass through a dropout layer
        out = self.dropout(r_output)

        # Stack up LSTM outputs using view
        # you may need to use contiguous to reshape the output
        out = out.contiguous().reshape(-1, self.n_hidden)

        ## TODO: put x through the fully-connected layer
        out = self.fc(out)

        # return the final output and the hidden state
        return out, hidden

    def init_hidden(self, batch_size):
        """Initializes hidden state"""
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data

        if train_on_gpu:
            hidden = (
                weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
            )
        else:
            hidden = (
                weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
            )

        return hidden

In [11]:
def train(
    net,
    data,
    epochs=10,
    batch_size=10,
    seq_length=50,
    lr=0.001,
    clip=5,
    val_frac=0.1,
    print_every=10,
):
    """Training a network

    Arguments
    ---------

    net: CharRNN network
    data: text data to train the network
    epochs: Number of epochs to train
    batch_size: Number of mini-sequences per mini-batch, aka batch size
    seq_length: Number of character steps per mini-batch
    lr: learning rate
    clip: gradient clipping
    val_frac: Fraction of data to hold out for validation
    print_every: Number of steps for printing training and validation loss

    """
    net.train()

    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    # create training and validation data
    val_idx = int(len(data) * (1 - val_frac))
    data, val_data = data[:val_idx], data[val_idx:]

    if train_on_gpu:
        net.cuda()

    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)

        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1

            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)

            if train_on_gpu:
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()

            # get the output from the model
            output, h = net(inputs, h)

            # calculate the loss and perform backprop
            loss = criterion(output, targets.reshape(batch_size * seq_length).long())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()

            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)

                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])

                    inputs, targets = x, y
                    if train_on_gpu:
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(
                        output, targets.reshape(batch_size * seq_length).long()
                    )

                    val_losses.append(val_loss.item())

                net.train()  # reset to train mode after iterationg through validation data

                print(
                    "Epoch: {}/{}...".format(e + 1, epochs),
                    "Step: {}...".format(counter),
                    "Loss: {:.4f}...".format(loss.item()),
                    "Val Loss: {:.4f}".format(np.mean(val_losses)),
                )

In [12]:
# define and print the net
n_hidden = 512
n_layers = 2

net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(156, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=156, bias=True)
)


In [13]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()

if train_on_gpu:
    print("Training on GPU!")
else:
    print("No GPU available, training on CPU; consider making n_epochs very small.")

No GPU available, training on CPU; consider making n_epochs very small.


In [14]:
net

CharRNN(
  (lstm): LSTM(156, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=156, bias=True)
)

In [15]:
batch_size = 128
seq_length = 100
n_epochs = 10  # start smaller if you are just testing initial behavior

# train the model
train(
    net,
    encoded,
    epochs=n_epochs,
    batch_size=batch_size,
    seq_length=seq_length,
    lr=0.001,
    print_every=10,
)

Epoch: 1/10... Step: 10... Loss: 3.5150... Val Loss: 3.3983
Epoch: 1/10... Step: 20... Loss: 3.4107... Val Loss: 3.3299
Epoch: 1/10... Step: 30... Loss: 3.3422... Val Loss: 3.3146
Epoch: 1/10... Step: 40... Loss: 3.3981... Val Loss: 3.3154
Epoch: 1/10... Step: 50... Loss: 3.3429... Val Loss: 3.3128
Epoch: 1/10... Step: 60... Loss: 3.3894... Val Loss: 3.3088
Epoch: 1/10... Step: 70... Loss: 3.3300... Val Loss: 3.3041
Epoch: 1/10... Step: 80... Loss: 3.3051... Val Loss: 3.2880
Epoch: 1/10... Step: 90... Loss: 3.2946... Val Loss: 3.2541
Epoch: 1/10... Step: 100... Loss: 3.2446... Val Loss: 3.2048
Epoch: 1/10... Step: 110... Loss: 3.1767... Val Loss: 3.1374
Epoch: 1/10... Step: 120... Loss: 3.1268... Val Loss: 3.0705
Epoch: 2/10... Step: 130... Loss: 3.0352... Val Loss: 2.9818
Epoch: 2/10... Step: 140... Loss: 2.8912... Val Loss: 2.8707
Epoch: 2/10... Step: 150... Loss: 2.8228... Val Loss: 2.7722
Epoch: 2/10... Step: 160... Loss: 2.7333... Val Loss: 2.7040
Epoch: 2/10... Step: 170... Loss:

In [16]:
# change the name, for saving multiple files
model_name = "rnn_x_epoch.net"

checkpoint = {
    "n_hidden": net.n_hidden,
    "n_layers": net.n_layers,
    "state_dict": net.state_dict(),
    "tokens": net.chars,
}

with open(model_name, "wb") as f:
    torch.save(checkpoint, f)

In [17]:
def predict(net, char, h=None, top_k=None):
    """Given a character, predict the next character.
    Returns the predicted character and the hidden state.
    """

    # tensor inputs
    x = np.array([[net.char2int[char]]])
    x = one_hot_encode(x, len(net.chars))
    inputs = torch.from_numpy(x)

    if train_on_gpu:
        inputs = inputs.cuda()

    # detach hidden state from history
    h = tuple([each.data for each in h])
    # get the output of the model
    out, h = net(inputs, h)

    # get the character probabilities
    p = F.softmax(out, dim=1).data
    if train_on_gpu:
        p = p.cpu()  # move to cpu

    # get top characters
    if top_k is None:
        top_ch = np.arange(len(net.chars))
    else:
        p, top_ch = p.topk(top_k)
        top_ch = top_ch.numpy().squeeze()

    # select the likely next character with some element of randomness
    p = p.numpy().squeeze()
    char = np.random.choice(top_ch, p=p / p.sum())

    # return the encoded value of the predicted char and the hidden state
    return net.int2char[char], h

In [18]:
def sample(net, size, prime="The", top_k=None):

    if train_on_gpu:
        net.cuda()
    else:
        net.cpu()

    net.eval()  # eval mode
#     
#     print("!!!!!!!!!!!!!")
    total_params = sum(p.numel() for p in net.parameters())
#     print(total_params)
#     for laeyrs in net.parameters():
#         print(laeyrs.size())
#         print("\n")
#     print("!!!!!!!!!!!!!")

    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)

    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return "".join(chars)

In [19]:
print(sample(net, 1000, prime="Harry", top_k=5))

!!!!!!!!!!!!!
3553436
!!!!!!!!!!!!!
Harryats




Гарри подумал на серяденных вопросы напостоянние.



— Во возможносте всё, — подумал, как правложить в кому-то столака, — теперь неприметли серьдо проста ило с крикного ответили. — Нить, которые предлижалась, как положение, что превсалась. Но подумайте, котырует приделать то же стало вы высоли, предвольши никогда не могу придётать. Но то сказал, что ты со настоется не сметра, начаственно объятнельне подумал.



Гурри всё в таким пломого в своего открыла, но откролятся потому, и не посметил себе профессору Квирреллу. — На предпередивений положить тебя на столу, пока они понял, что надолженных. На на могут подумальтете в как всех прочем, но она не было одобалось в мог по верноваться, к того, чтобы произнесла отверить пальци, и он пристольсла потому и не всем случилось, не произниса, чем она не подумала и посколько на подемятся.



— Могло он всё так. Профессор Квиррелл вы напринименный ваши, — потому что теперь, к не смысли посметреть, что